# Using Pandas to change data

In the last module we discussed relational data and Dataframes then investigated how to investigate and select data in a Dataframe.  In this module we will show you how to change a Dataframe to add data, delete data, and transform data.

<div style="padding: 5px; padding-left: 10px;">
    
<center><h2>Table of Contents</h2></center>

<h4><a href='#manipulate'>Table & Data Manipulation</a></h4>
<li>Adjust column types</li>
<li>Add/create new columns
<li>Drop columns
<li>Sort values
  
<h4><a href='#plot'>Table transformation, aggregation & plotting</a></h4>
<li>Wide vs long tables
<li>Melt (transform wide to long df)
<li>Pivot (transform long to wide df)
<li>Cross tab (aggregate counts)
<li>Pivot_table (flexible aggregation)
<li>Groupby & agg

<h4><a href='#table'>Create new tables, join tables, save tables</a></h4>
<li>Join & merge tables
<li>Create dataframe from scratch
<li>Saving dataframes 



**Import libraries and data**    
Here we'll load in the libraries we'll use to shape and explore the data and the data itself.


In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
filepath = os.path.join('support_files', 'datasets', 'messy_superstore_data.feather')
df = pd.read_feather(filepath)

# ignore this line of code for now- it will be explained a bit later
df.set_index('Row ID', inplace = True)

<a id='manipulate'></a>

## Data Manipulations
    
* <a href='#astype'> Adjust column type</a> 
* <a href='#addcolumns'> Create new columns</a>
    * <a href='#apply'> .apply()</a>
* <a href='#dropcolumns'>Drop unneccesary or redundant columns</a>
* <a href='#dropna'>Drop nan, NULL entries</a>
* <a href='#sort_values'>Sort the whole dataframe based on one column's values</a>


<a id='astype'></a>

#### Change column data type
    
After examining your dataframe you may find that there are certain columns whose data types do not match your assumptions or whose data types could inhibit analysis. For example, if you have a column that contains numbers, but the data type is string- you would not actually be able to do numeric operations on it. You can cast the data to a different datatype using  **[.astype()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)**. 
    
To cast a column to a new data type: 
<code>df['column_1'] = df['column_1'].astype(new type)</code>

Remember some of the possible datatypes are:
* int
* float64
* bool
* string

In [5]:
# we know that in our dataframe quantity should be an integer 
# but right now it's a float- lets change that using .astype()
# then we can check the type after

# change the type to an int
df['Quantity'] = df['Quantity'].astype(int)

# check the data type
df["Quantity"]

Row ID
IN-2014-23218    2
IN-2014-24599    2
IN-2014-24597    4
IN-2014-27993    5
IN-2014-28967    7
                ..
ZA-2014-49187    1
ZI-2014-42069    1
ZI-2014-43712    1
ZI-2014-48372    2
ZI-2014-48014    1
Name: Quantity, Length: 17531, dtype: int64

<a id='addcolumns'></a>

#### Add new columns
    
Frequently you'll want to add **[new columns](https://pandas.pydata.org/docs/getting_started/intro_tutorials/05_add_columns.html)**. There are multiple ways to do this! 

<img src="../support_files/images/pandas/pandas_make_new_columns.png">    
To add already computed data:
* <code>df['new_column_name'] = already_computed_data
    
You can also calculate new columns based on currently existing columns: 
* <code>df['AB_SUM'] = df['A'] + df['B']</code>
* <code>df['Volume'] = df['Length'] * df['Height'] * df['Depth']</code>
    
Note: newly created columns are always added to the end of the dataframe    
</div>


In [6]:
# lets create a "Profit Per Unit" column by diving profit by quantity
df['Profit Per Unit'] = df['Profit'] / df['Quantity']

# now lets view the results
df[["Profit", "Quantity", "Profit Per Unit"]]

,Profit,Quantity,Profit Per Unit
Row ID,,,
IN-2014-23218,39.840,2,19.920
IN-2014-24599,102.420,2,51.210
IN-2014-24597,79.680,4,19.920
IN-2014-27993,848.700,5,169.740
IN-2014-28967,356.580,7,50.940
...,...,...,...
ZA-2014-49187,19.710,1,19.710
ZI-2014-42069,-20.799,1,-20.799
ZI-2014-43712,-59.562,1,-59.562


<a id='apply'></a>

#### .apply()
    
**[.apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)** is a powerful method that can do many things. In this context we will use it to create new columns using functions from other packages or functions you've created.

* <code>df['new_column_name'] = df[column(s) of interest].apply(function)</code>
    
* <code>df['A_sqrt'] = df['A'].apply(np.sqrt)</code>
* <code>df['A_adjusted'] = df['A'].apply(your_function)</code>

In [7]:
# lets practice by making a function that will fix the capitolization of the 
# Category" column and then applying it

def capitalize_first_letter(string):
    all_lower = string.lower() 
    cap_first = all_lower.capitalize()
    return cap_first

df['Category'] = df['Category'].apply(capitalize_first_letter)

# view the results
df[['Category']]

,Category
Row ID,
IN-2014-23218,Furniture
IN-2014-24599,Furniture
IN-2014-24597,Furniture
IN-2014-27993,Furniture
IN-2014-28967,Furniture
...,...
ZA-2014-49187,Technology
ZI-2014-42069,Technology
ZI-2014-43712,Technology


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**Exercise 8.1**: In these code examples, what do you think the <code>axis</code> argument is doing?
What kind of output do you think will be returned? for each? 


* <code>df[['A', 'B']].apply(np.mean, axis = 0)</code>
* <code>df[['A', 'B']].apply(np.mean, axis = 1)</code>
    
**Exercise 8.2**: Which axis setting is the default? (hint- check the documentation)
</div>

**Answer 7.1:** 

* 0 or ‘index’: apply function to each column.
* 1 or ‘columns’: apply function to each row.

In [10]:
# code to see the output differences 
df[["Quantity", "Profit Per Unit"]].apply(np.sum, axis = 0)

Quantity            60622.00000
Profit Per Unit    144656.52516
dtype: float64

In [11]:
# code to see the output differences 
df[["Quantity", "Profit Per Unit"]].apply(np.sum, axis = 1)

Row ID
IN-2014-23218     21.920
IN-2014-24599     53.210
IN-2014-24597     23.920
IN-2014-27993    174.740
IN-2014-28967     57.940
                  ...   
ZA-2014-49187     20.710
ZI-2014-42069    -19.799
ZI-2014-43712    -58.562
ZI-2014-48372    -44.590
ZI-2014-48014    -19.298
Length: 17531, dtype: float64

**Answer 7.2** 

0 is the default- meaning it will automatically apply to columns.

<a id='lambda'></a>

#### lambda functions


Python Lambda Functions are anonymous functions.  This means that the function is without a name. As we already know that the def keyword is used to define a normal function in Python. Similarly, the lambda keyword is used to define an anonymous function in Python.

* syntactically restricted to a single expression
* Good for performing short operations/data manipulations    
* generally reduce the readability of code  

A [lambda](https://www.geeksforgeeks.org/python-lambda-anonymous-functions-filter-map-reduce/#) function is often used with .apply() for pandas data manipulations.
    
* <code>lambda string: string.lower().capitalize() </code>   
    
        
Defined function: 

<code>def cube(num):
    return num * num * num
</code> 

Lambda equivalent:

<code>lambda num: num * num * num</code>


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise 8.3**: create a lambda function that adds 100 to an input value

</div>

In [12]:
# Answer 7.3
plus_100 = lambda num: num + 100

#### Using .apply(lambda)

**[.apply(lambda)]( https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/#)** is frequently used for row-wise operations, though there are several ways it can be utilized. 

In this setting lambda is the function that will be used by .apply
* <code>data.apply(lambda input: manipulation) </code>

Column-wise generic example: 
* <code>df['text column'].apply(lambda string: string.lower().capitalize()) </code>
    
Row-wise generic example: 
* <code>df.apply(lambda row: df['column_A']*df['column_B'], axis = 1) </code>      


In [13]:
# lets use .apply(lambda) to create a new column that will count the number of manufactuters 
# from the 'manufacturers' column array

df['manufacturers_count'] = df.apply(lambda row : len(row['manufacturers']), axis = 1)

In [14]:
df[['Category', 'Sub-Category','Product Name', 'manufacturers', 'manufacturers_count']]

,Category,Sub-Category,Product Name,manufacturers,manufacturers_count
Row ID,,,,,
IN-2014-23218,Furniture,FURNISHINGS,"Rubbermaid Door Stop, Erganomic","[Dunder Mifflin, Globex Corp, Hudsucker Indust...",5
IN-2014-24599,Furniture,BOOKCASES,"Ikea Library with Doors, Mobile","[ACME Co, Buy n Large, Dunder Mifflin, Globex ...",6
IN-2014-24597,Furniture,FURNISHINGS,"Rubbermaid Door Stop, Erganomic","[Dunder Mifflin, LexCorp, Olivander Crafts, Ro...",5
IN-2014-27993,Furniture,BOOKCASES,"Bush Classic Bookcase, Pine","[Dunder Mifflin, Olivander Crafts]",2
IN-2014-28967,Furniture,CHAIRS,"Hon Rocking Chair, Red","[ACME Co, Buy n Large, Dunder Mifflin, LexCorp...",6
...,...,...,...,...,...
ZA-2014-49187,Technology,ACCESSORIES,"Memorex Router, USB","[ACME Co, Royco Waystar, Umbrella Corporation]",3
ZI-2014-42069,Technology,MACHINES,"StarTech Phone, Red",[Dunder Mifflin],1
ZI-2014-43712,Technology,ACCESSORIES,"Belkin Router, USB","[ACME Co, Buy n Large, Hudsucker Industries]",3


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Execise 7.4**: Use <code>.apply(lambda )</code> to apply the <code>capitalize_first_letter function</code> to the 'Sub-Category' column

</div>

In [15]:
# Answer 7.4
df['Sub-Category'] = df['Sub-Category'].apply(lambda string: capitalize_first_letter(string)) 
df['Sub-Category']

Row ID
IN-2014-23218    Furnishings
IN-2014-24599      Bookcases
IN-2014-24597    Furnishings
IN-2014-27993      Bookcases
IN-2014-28967         Chairs
                    ...     
ZA-2014-49187    Accessories
ZI-2014-42069       Machines
ZI-2014-43712    Accessories
ZI-2014-48372       Machines
ZI-2014-48014       Machines
Name: Sub-Category, Length: 17531, dtype: object

<a id='dropcolumns'></a>

#### Drop specific columns & rows using</h4> 
    
You can easily drop rows or columns using **<code>[.drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)</code>**

There are many reasons you may wish to drop columns from your dataframe. Here are a few examples: 
* irrlevant to the analysis you are going to do
* out of date
* redundant information
* non-analyzable due to high rates of Nan or empty entries
    
Drop columns:    
* <code>df = df.drop(columns=['column1', 'columns2'])</code>
* <code>df = df.drop('column_name',axis=1)</code>

Drop rows:
* <code>df = df.drop(index=('index1_string, 'index2_string'))</code>   
* <code>df = df.drop([row, row])</code>


In [16]:
# Lets drop the 'Category (OLD)'column 
 
df.drop(columns=['Category (OLD)'], inplace=True)

# check the results in the dataframe columns list
df.columns

Index(['Order ID', 'Segment', 'Category', 'Sub-Category', 'Product Name',
       'Product ID', 'Country', 'Market', 'Region', 'Quantity', 'Discount',
       'Profit', 'Customer ID', 'Customer Name', 'Order Priority',
       'Postal Code', 'Ship Mode', 'Shipping Cost', '10/1/2014', '7/1/2014',
       '11/1/2014', '9/1/2014', '1/1/2014', '12/1/2014', '8/1/2014',
       '5/1/2014', '3/1/2014', '4/1/2014', '2/1/2014', '6/1/2014', 'City',
       'State', 'manufacturers', 'Profit Per Unit', 'manufacturers_count'],
      dtype='object')

#### Modifying data inplace

**<code>[inplace](https://towardsdatascience.com/why-you-should-probably-never-use-pandas-inplace-true-9f9f211849e4#:~:text=Using%20the%20inplace%3DTrue%20keyword,which%20you%20apply%20it%20to.)</code>** is a parameter accepted by a number of pandas methods which affects the behaviour of how the method runs.
Some examples of where you might commonly see this keyword are the methods(non-exhaustive): 
* <code>[.drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)</code>
* <code>[.fillna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)</code>
* <code>[.replace()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)</code>
* <code>[.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)</code>


Using the <code>inplace=True</code> keyword in a pandas method changes the default behaviour such that the operation on the dataframe doesn’t return anything, it instead ‘modifies the underlying data’. It mutates the actual object which you apply it to.


<a id='dropna'></a>

#### remove rows with missing values

For columns of particular importance may wish to exclude rows that have missing values to do this we use 
**<code>[.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)</code>**

Example: 
    <code>df.dropna(subset=['column'], inplace=bool)</code>


In [17]:
df.shape

(17531, 35)

In [18]:
df.dropna(subset=['Market'], inplace=True)

In [19]:
# There are no na in the Market column so the shape is the same
df.shape

(17531, 35)

<a id='sort_values'></a>

#### Sort the dataframe based on column values
    
**<code>[.sort_values()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)</code>** to sort your dataframe by the the values of a particular column or columns. 

<code>df.sort_values(by=['column1', 'column2'], ascending = boolean, inplace= boolean)</code>
    
Pro-tip: Sorting the values of your dataframe before plotting creates more interpretable plots. 


In [20]:
# Lets sort our df by 'Segment', 'Category' and 'Sub-Category' 
# we will leave the underlying data unchanged by setting inpace to False

df.sort_values(by = ['Segment', 'Category', 'Sub-Category'], inplace = False)

,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,Quantity,...,5/1/2014,3/1/2014,4/1/2014,2/1/2014,6/1/2014,City,State,manufacturers,Profit Per Unit,manufacturers_count
Row ID,,,,,,,,,,,,,,,,,,,,,
AG-2014-46003,AG-2014-8600,Consumer,Furniture,Bookcases,"Sauder Classic Bookcase, Pine",FUR-SAU-10002255,Algeria,Africa,Africa,2,...,NaN,NaN,NaN,NaN,NaN,Batna,Batna,"[ACME Co, Globex Corp, Umbrella Corporation]",74.3400,3
AO-2014-45199,AO-2014-9370,Consumer,Furniture,Bookcases,"Bush Classic Bookcase, Pine",FUR-BUS-10001243,Angola,Africa,Africa,2,...,828.06,NaN,NaN,NaN,NaN,Luanda,Luanda,"[ACME Co, Buy n Large, Globex Corp, Hudsucker ...",202.8600,7
US-2014-6344,US-2014-165512,Consumer,Furniture,Bookcases,"Bush Library with Doors, Metal",FUR-BO-10002968,Argentina,LATAM,South,2,...,NaN,291.0240,NaN,NaN,NaN,Buenos Aires,Buenos Aires,"[Dunder Mifflin, Globex Corp, Hudsucker Indust...",-58.2080,6
US-2014-1158,US-2014-107664,Consumer,Furniture,Bookcases,"Ikea Floating Shelf Set, Traditional",FUR-BO-10001585,Argentina,LATAM,South,4,...,NaN,NaN,NaN,NaN,NaN,La Plata,Provincia de Buenos Aires,"[Dunder Mifflin, Hudsucker Industries, Royco W...",-13.6440,3
IN-2014-21802,IN-2014-25644,Consumer,Furniture,Bookcases,"Bush Corner Shelving, Metal",FUR-BO-10004230,Australia,APAC,Oceania,4,...,NaN,NaN,NaN,NaN,NaN,Tamworth,New South Wales,"[Hudsucker Industries, LexCorp]",38.2650,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-40156,US-2014-114356,Home Office,Technology,Phones,Plantronics Encore H101 Dual Earpieces Headset,TEC-PH-10003171,United States,US,South,1,...,NaN,NaN,NaN,NaN,NaN,Charlotte,North Carolina,"[ACME Co, Buy n Large, LexCorp, Olivander Craf...",3.5960,6
US-2014-33227,US-2014-101539,Home Office,Technology,Phones,Mitel MiVoice 5330e IP Phone,TEC-PH-10004165,United States,US,West,3,...,NaN,659.9760,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Royco Waystar, Umbrella Corporation,...",16.4994,4
CA-2014-36882,CA-2014-105333,Home Office,Technology,Phones,Panasonic Business Telephones KX-T7736,TEC-PH-10001468,United States,US,East,3,...,NaN,NaN,NaN,NaN,NaN,New York City,New York,"[LexCorp, Umbrella Corporation, Wayne Enterpri...",54.6060,3


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise 8.5**: sort the table by profit per unit descending

**Exercise 8.6**: sort the table by profit per unit ascending
</div>


In [21]:
# Answer 7.5
df.sort_values(by = 'Profit Per Unit', inplace=False, ascending=False)

,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,Quantity,...,5/1/2014,3/1/2014,4/1/2014,2/1/2014,6/1/2014,City,State,manufacturers,Profit Per Unit,manufacturers_count
Row ID,,,,,,,,,,,,,,,,,,,,,
CA-2014-35487,CA-2014-166709,Consumer,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,TEC-CO-10004722,United States,US,East,3,...,NaN,NaN,NaN,NaN,NaN,Newark,Delaware,"[Buy n Large, Dunder Mifflin, Globex Corp, Hud...",1679.9952,8
CA-2014-39450,CA-2014-140151,Consumer,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,TEC-CO-10004722,United States,US,West,4,...,NaN,13999.96,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Royco Waystar]",1679.9952,2
CA-2014-33920,CA-2014-127180,Home Office,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,TEC-CO-10004722,United States,US,East,4,...,NaN,NaN,NaN,NaN,NaN,New York City,New York,[Globex Corp],979.9972,1
CA-2014-37817,CA-2014-138289,Consumer,Office supplies,Binders,GBC DocuBind P400 Electric Binding System,OFF-BI-10004995,United States,US,Central,4,...,NaN,NaN,NaN,NaN,NaN,Jackson,Michigan,"[Buy n Large, Hudsucker Industries, LexCorp, O...",626.0554,6
SF-2014-48905,SF-2014-4490,Consumer,Furniture,Tables,"Bevis Conference Table, Rectangular",FUR-BEV-10004805,South Africa,Africa,Africa,2,...,NaN,NaN,NaN,NaN,NaN,George,Western Cape,[Dunder Mifflin],442.2300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CA-2014-34143,CA-2014-152093,Home Office,Office supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,OFF-BI-10003527,United States,US,Central,3,...,NaN,NaN,NaN,NaN,NaN,Chicago,Illinois,"[Buy n Large, Dunder Mifflin, LexCorp]",-381.2970,3
CA-2014-36607,CA-2014-131254,Consumer,Office supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,OFF-BI-10003527,United States,US,Central,6,...,NaN,NaN,NaN,NaN,NaN,Houston,Texas,"[Buy n Large, LexCorp]",-381.2970,2
US-2014-36288,US-2014-122714,Corporate,Office supplies,Binders,Ibico EPK-21 Electric Binding System,OFF-BI-10001120,United States,US,Central,5,...,NaN,NaN,NaN,NaN,NaN,Chicago,Illinois,[Umbrella Corporation],-585.8969,1


In [22]:
# Answer 7.6
df.sort_values(by = 'Profit Per Unit', inplace=False, ascending=True)

,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,Quantity,...,5/1/2014,3/1/2014,4/1/2014,2/1/2014,6/1/2014,City,State,manufacturers,Profit Per Unit,manufacturers_count
Row ID,,,,,,,,,,,,,,,,,,,,,
US-2014-31980,US-2014-168116,Corporate,Technology,Machines,Cubify CubeX 3D Printer Triple Head Print,TEC-MA-10004125,United States,US,South,4,...,NaN,NaN,NaN,NaN,NaN,Burlington,North Carolina,"[ACME Co, Buy n Large, Dunder Mifflin, Globex ...",-959.9976,7
CA-2014-34308,CA-2014-134845,Home Office,Technology,Machines,Lexmark MX611dhe Monochrome Laser Printer,TEC-MA-10000822,United States,US,West,5,...,NaN,NaN,2549.985,NaN,NaN,Louisville,Colorado,"[Hudsucker Industries, LexCorp, Olivander Craf...",-679.9960,4
US-2014-36288,US-2014-122714,Corporate,Office supplies,Binders,Ibico EPK-21 Electric Binding System,OFF-BI-10001120,United States,US,Central,5,...,NaN,NaN,NaN,NaN,NaN,Chicago,Illinois,[Umbrella Corporation],-585.8969,1
CA-2014-36607,CA-2014-131254,Consumer,Office supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,OFF-BI-10003527,United States,US,Central,6,...,NaN,NaN,NaN,NaN,NaN,Houston,Texas,"[Buy n Large, LexCorp]",-381.2970,2
CA-2014-34143,CA-2014-152093,Home Office,Office supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,OFF-BI-10003527,United States,US,Central,3,...,NaN,NaN,NaN,NaN,NaN,Chicago,Illinois,"[Buy n Large, Dunder Mifflin, LexCorp]",-381.2970,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF-2014-48905,SF-2014-4490,Consumer,Furniture,Tables,"Bevis Conference Table, Rectangular",FUR-BEV-10004805,South Africa,Africa,Africa,2,...,NaN,NaN,NaN,NaN,NaN,George,Western Cape,[Dunder Mifflin],442.2300,1
CA-2014-37817,CA-2014-138289,Consumer,Office supplies,Binders,GBC DocuBind P400 Electric Binding System,OFF-BI-10004995,United States,US,Central,4,...,NaN,NaN,NaN,NaN,NaN,Jackson,Michigan,"[Buy n Large, Hudsucker Industries, LexCorp, O...",626.0554,6
CA-2014-33920,CA-2014-127180,Home Office,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,TEC-CO-10004722,United States,US,East,4,...,NaN,NaN,NaN,NaN,NaN,New York City,New York,[Globex Corp],979.9972,1


<a id='plot'></a>

## Table transformation, aggregation & Plotting
    
* <a href='#wide_vs_long'>Wide vs Long tables</a>
* <a href='#melt'>Transform to a wide dataframe using .melt</a>
* <a href='#pivot'>Transform to a long dataframe using .pivot</a>

* <a href='#crosstab'> Get a count of categorical variables with crosstab</a>   
* <a href='#groupby'> Aggregate data using .groupby and and get sum using .agg</a>
* <a href='#pivot_table'> Flexible aggregation of categorical variables with .pivot_tabe</a>    

NOTE: while this tutorial uses pandas built in plotting functions there are many better packages for plotting, the [Seaborn](https://seaborn.pydata.org/) package works especially well with pandas dataframe and other tabular style data.  We will cover Seaborn in a later module.


In [23]:
# lets create a new dataframe for this next set of examples
# use .loc[] to select the united states as the country and .copy() to ensure 
# it's a new object and not altering our base dataframe

USA_df = df.loc[df["Country"]=="United States"].copy()
USA_df

,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,Quantity,...,5/1/2014,3/1/2014,4/1/2014,2/1/2014,6/1/2014,City,State,manufacturers,Profit Per Unit,manufacturers_count
Row ID,,,,,,,,,,,,,,,,,,,,,
CA-2014-39465,CA-2014-107174,Home Office,Furniture,Tables,Hon 5100 Series Wood Tables,FUR-TA-10004575,United States,US,West,7,...,NaN,NaN,NaN,NaN,NaN,Seattle,Washington,"[Globex Corp, Umbrella Corporation, Wayne Ente...",52.3764,3
CA-2014-39466,CA-2014-107174,Home Office,Furniture,Chairs,Hon 2090 “Pillow Soft” Series Mid Back Swivel/...,FUR-CH-10003312,United States,US,West,2,...,NaN,NaN,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Buy n Large, Globex Corp, Hudsucker ...",-36.5274,6
CA-2014-33848,CA-2014-101042,Consumer,Furniture,Furnishings,"3M Polarizing Task Lamp with Clamp Arm, Light ...",FUR-FU-10004665,United States,US,South,6,...,NaN,NaN,NaN,NaN,NaN,Henderson,Kentucky,"[ACME Co, Globex Corp, Hudsucker Industries, L...",35.6148,5
CA-2014-32897,CA-2014-158876,Consumer,Furniture,Furnishings,Telescoping Adjustable Floor Lamp,FUR-FU-10001967,United States,US,Central,2,...,NaN,NaN,NaN,NaN,NaN,Carrollton,Texas,"[Dunder Mifflin, Globex Corp, Olivander Crafts]",-6.9965,3
CA-2014-39961,CA-2014-126634,Consumer,Furniture,Furnishings,Flat Face Poster Frame,FUR-FU-10004973,United States,US,West,5,...,NaN,94.200,NaN,NaN,NaN,Lakewood,California,"[Olivander Crafts, Umbrella Corporation]",7.9128,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-33227,US-2014-101539,Home Office,Technology,Phones,Mitel MiVoice 5330e IP Phone,TEC-PH-10004165,United States,US,West,3,...,NaN,659.976,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Royco Waystar, Umbrella Corporation,...",16.4994,4
CA-2014-36882,CA-2014-105333,Home Office,Technology,Phones,Panasonic Business Telephones KX-T7736,TEC-PH-10001468,United States,US,East,3,...,NaN,NaN,NaN,NaN,NaN,New York City,New York,"[LexCorp, Umbrella Corporation, Wayne Enterpri...",54.6060,3
CA-2014-40289,CA-2014-159100,Corporate,Technology,Accessories,Memorex Mini Travel Drive 8 GB USB 2.0 Flash D...,TEC-AC-10003499,United States,US,South,2,...,NaN,NaN,NaN,NaN,NaN,Columbia,Tennessee,"[Buy n Large, Dunder Mifflin, LexCorp]",2.2002,3


#### Resetting the index
    
In many of the table transformations, the process of transforming the data changes the index. If we wish to use the index row we will want to reset the index so that it starts at 0 and increases sequentially before we do these transformations so it is saved in a regular column format. 
    
This is easily achievable with **<code>[.reset_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html)<code>**. 

 * Use <code>drop=True</code> if it's not necessary to maintain a column with the original indexes.


In [25]:
USA_df.reset_index(inplace=True)

USA_df

,index,Row ID,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,...,5/1/2014,3/1/2014,4/1/2014,2/1/2014,6/1/2014,City,State,manufacturers,Profit Per Unit,manufacturers_count
0,0,CA-2014-39465,CA-2014-107174,Home Office,Furniture,Tables,Hon 5100 Series Wood Tables,FUR-TA-10004575,United States,US,...,NaN,NaN,NaN,NaN,NaN,Seattle,Washington,"[Globex Corp, Umbrella Corporation, Wayne Ente...",52.3764,3
1,1,CA-2014-39466,CA-2014-107174,Home Office,Furniture,Chairs,Hon 2090 “Pillow Soft” Series Mid Back Swivel/...,FUR-CH-10003312,United States,US,...,NaN,NaN,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Buy n Large, Globex Corp, Hudsucker ...",-36.5274,6
2,2,CA-2014-33848,CA-2014-101042,Consumer,Furniture,Furnishings,"3M Polarizing Task Lamp with Clamp Arm, Light ...",FUR-FU-10004665,United States,US,...,NaN,NaN,NaN,NaN,NaN,Henderson,Kentucky,"[ACME Co, Globex Corp, Hudsucker Industries, L...",35.6148,5
3,3,CA-2014-32897,CA-2014-158876,Consumer,Furniture,Furnishings,Telescoping Adjustable Floor Lamp,FUR-FU-10001967,United States,US,...,NaN,NaN,NaN,NaN,NaN,Carrollton,Texas,"[Dunder Mifflin, Globex Corp, Olivander Crafts]",-6.9965,3
4,4,CA-2014-39961,CA-2014-126634,Consumer,Furniture,Furnishings,Flat Face Poster Frame,FUR-FU-10004973,United States,US,...,NaN,94.200,NaN,NaN,NaN,Lakewood,California,"[Olivander Crafts, Umbrella Corporation]",7.9128,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314,3314,US-2014-33227,US-2014-101539,Home Office,Technology,Phones,Mitel MiVoice 5330e IP Phone,TEC-PH-10004165,United States,US,...,NaN,659.976,NaN,NaN,NaN,Seattle,Washington,"[ACME Co, Royco Waystar, Umbrella Corporation,...",16.4994,4
3315,3315,CA-2014-36882,CA-2014-105333,Home Office,Technology,Phones,Panasonic Business Telephones KX-T7736,TEC-PH-10001468,United States,US,...,NaN,NaN,NaN,NaN,NaN,New York City,New York,"[LexCorp, Umbrella Corporation, Wayne Enterpri...",54.6060,3
3316,3316,CA-2014-40289,CA-2014-159100,Corporate,Technology,Accessories,Memorex Mini Travel Drive 8 GB USB 2.0 Flash D...,TEC-AC-10003499,United States,US,...,NaN,NaN,NaN,NaN,NaN,Columbia,Tennessee,"[Buy n Large, Dunder Mifflin, LexCorp]",2.2002,3
3317,3317,CA-2014-36007,CA-2014-138149,Consumer,Technology,Accessories,Enermax Briskie RF Wireless Keyboard and Mouse...,TEC-AC-10001284,United States,US,...,NaN,NaN,NaN,NaN,62.31,Los Angeles,California,"[Royco Waystar, Umbrella Corporation]",7.4772,2


<a id='wide_vs_long'></a>

#### Wide vs Long dataframes

<img src="../support_files/images/pandas/pandas_wide_vs_long_df_b.png" width=500/> 

**Wide**


* Methods for creating: <code>df.pivot()</code> and <code>pd.pivot_table(df)</code> can reshape a long dataframe into a wide dataframe
    
**Long**
 

* Methods for creating: <code>pd.melt(df)</code> can reshape a wide dataframe into a long dataframe


<a id='melt'></a>

#### create a long dataframe with .melt()

**<code>[.melt()](https://pandas.pydata.org/docs/reference/api/pandas.melt.html)</code>** gathers columns into rows. 

It can be especially effective if you have several columns related columns containing many NaNs that would be better represented (and reduce the number of NaNs) if gathered into rows

<img src="../support_files/images/pandas/pandas_melt.png">  
        

<code>df_melt = pd.melt(df, id_vars = [ columns ], value_vars= [ columns ], var_name = string, value_name = string)</code>
    
* <code>id_vars</code>: Column(s) to use as identifier variables
    * columns you want to retain/ keep intact)
* <code>value_vars</code>: Column(s) to unpivot. If not specified, uses all columns that are not set as id_vars.
    * columns to transform
* <code>var_name</code>:  Name to use for the ‘variable’ column. If None it uses frame.columns.name or ‘variable’.
    * what are the columns you want to transform (are they cities? dates? something else?
* <code>value_name</code>: Name to use for the ‘value’ column. 
    * what are the values of the columns you are going to transform (cost? population? temperature? etc?)
    
<img src="../support_files/images/pandas/pandas_melt_b.png" width ='60%'>




In our next example we want to transform our data from wide to long so we can concentrate NaNs to a single column to deal with removing them

<img src="support_files/images/pandas/pandas_melt_practical_example.png" width ='60%'>


In [38]:
# You may have noticed that there are many missing values in the columns 
# that contain each month, ie "1/1/2014". 
# We will gather these values from columns to rows, creating two new columns

# the variables are the dates and the values represent sales


# the column labels (var_name) all represent order date
# the values(value_name) represent sales

# try to keep your formatting legible by doing like so:
columns_to_keep = ['Row ID','Order ID', 'Segment', 'Category', 'Sub-Category', 'Product Name',
                   'Product ID', 'Country', 'Market', 'Region', 'City', 'State', 
                   'Quantity', 'Discount', 'Profit', 'Customer ID', 'Customer Name',
                    'Order Priority', 'Postal Code', 'Ship Mode', 'Shipping Cost']


columns_to_transform = ['10/1/2014', '7/1/2014', '11/1/2014', '9/1/2014', '1/1/2014',
                        '12/1/2014', '8/1/2014', '5/1/2014', '3/1/2014', '4/1/2014',
                        '2/1/2014', '6/1/2014']

USA_long = pd.melt(USA_df,
                   id_vars = columns_to_keep,
                   value_vars = columns_to_transform,
                   var_name = "Order Date",
                   value_name = "Sales")

In [39]:
# We can now see the new Dataframe, USA_long
USA_long

,Row ID,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,...,Discount,Profit,Customer ID,Customer Name,Order Priority,Postal Code,Ship Mode,Shipping Cost,Order Date,Sales
0,CA-2014-39465,CA-2014-107174,Home Office,Furniture,Tables,Hon 5100 Series Wood Tables,FUR-TA-10004575,United States,US,West,...,0.0,366.6348,AB-10060,Adam Bellavance,Low,98105.0,Standard Class,524.76,10/1/2014,NaN
1,CA-2014-39466,CA-2014-107174,Home Office,Furniture,Chairs,Hon 2090 “Pillow Soft” Series Mid Back Swivel/...,FUR-CH-10003312,United States,US,West,...,0.2,-73.0548,AB-10060,Adam Bellavance,Low,98105.0,Standard Class,47.81,10/1/2014,NaN
2,CA-2014-33848,CA-2014-101042,Consumer,Furniture,Furnishings,"3M Polarizing Task Lamp with Clamp Arm, Light ...",FUR-FU-10004665,United States,US,South,...,0.0,213.6888,AB-10105,Adrian Barton,Medium,42420.0,Standard Class,83.56,10/1/2014,NaN
3,CA-2014-32897,CA-2014-158876,Consumer,Furniture,Furnishings,Telescoping Adjustable Floor Lamp,FUR-FU-10001967,United States,US,Central,...,0.6,-13.9930,AB-10150,Aimee Bixby,Medium,75007.0,Second Class,1.54,10/1/2014,NaN
4,CA-2014-39961,CA-2014-126634,Consumer,Furniture,Furnishings,Flat Face Poster Frame,FUR-FU-10004973,United States,US,West,...,0.0,39.5640,AB-10165,Alan Barnes,Medium,90712.0,Second Class,4.69,10/1/2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39823,US-2014-33227,US-2014-101539,Home Office,Technology,Phones,Mitel MiVoice 5330e IP Phone,TEC-PH-10004165,United States,US,West,...,0.2,49.4982,VM-21685,Valerie Mitchum,High,98105.0,Second Class,77.64,6/1/2014,NaN
39824,CA-2014-36882,CA-2014-105333,Home Office,Technology,Phones,Panasonic Business Telephones KX-T7736,TEC-PH-10001468,United States,US,East,...,0.0,163.8180,VP-21730,Victor Preis,High,10011.0,Standard Class,64.39,6/1/2014,NaN
39825,CA-2014-40289,CA-2014-159100,Corporate,Technology,Accessories,Memorex Mini Travel Drive 8 GB USB 2.0 Flash D...,TEC-AC-10003499,United States,US,South,...,0.2,4.4004,VP-21760,Victoria Pisteka,Medium,38401.0,Standard Class,0.68,6/1/2014,NaN
39826,CA-2014-36007,CA-2014-138149,Consumer,Technology,Accessories,Enermax Briskie RF Wireless Keyboard and Mouse...,TEC-AC-10001284,United States,US,West,...,0.0,22.4316,WB-21850,William Brown,High,90049.0,First Class,22.06,6/1/2014,62.31


In [40]:
# drop rows with empty values in the 'Sales' column
usa_sales_long = USA_long.dropna(subset=['Sales'], inplace = False).copy()

In [41]:
# Now that we have our long dataframe
# Lets create a new column cost of goods "COGS" 
# we can calculate the cost of goods by by subtracting the profits from the sales
usa_sales_long['COGS'] = USA_long['Sales'] - USA_long['Profit']
usa_sales_long

,Row ID,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,...,Profit,Customer ID,Customer Name,Order Priority,Postal Code,Ship Mode,Shipping Cost,Order Date,Sales,COGS
18,CA-2014-39744,CA-2014-125451,Corporate,Furniture,Furnishings,Deflect-o DuraMat Antistatic Studded Beveled M...,FUR-FU-10000277,United States,US,East,...,50.5632,AH-10075,Adam Hart,Critical,2920.0,First Class,79.52,10/1/2014,210.680,160.1168
19,CA-2014-39743,CA-2014-125451,Corporate,Furniture,Furnishings,"Eldon 400 Class Desk Accessories, Black Carbon",FUR-FU-10004963,United States,US,East,...,14.7000,AH-10075,Adam Hart,Critical,2920.0,First Class,8.98,10/1/2014,35.000,20.3000
21,CA-2014-39742,CA-2014-125451,Corporate,Furniture,Tables,KI Adjustable-Height Table,FUR-TA-10001039,United States,US,East,...,-13.7568,AH-10075,Adam Hart,Critical,2920.0,First Class,68.01,10/1/2014,240.744,254.5008
22,CA-2014-39745,CA-2014-125451,Corporate,Furniture,Tables,"Office Impressions End Table, 20-1/2""H x 24""W ...",FUR-TA-10004915,United States,US,East,...,-127.5792,AH-10075,Adam Hart,Critical,2920.0,First Class,188.95,10/1/2014,637.896,765.4752
38,CA-2014-34336,CA-2014-101749,Corporate,Furniture,Tables,"Lesro Sheffield Collection Coffee Table, End T...",FUR-TA-10001520,United States,US,West,...,-6.4233,AS-10045,Aaron Smayling,Medium,91104.0,Standard Class,12.65,10/1/2014,171.288,177.7113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39801,CA-2014-32190,CA-2014-133256,Home Office,Technology,Phones,Nortel Networks T7316 E Nt8 B27,TEC-PH-10002660,United States,US,Central,...,135.9800,TH-21550,Tracy Hopkins,High,48227.0,First Class,171.35,6/1/2014,543.920,407.9400
39811,CA-2014-32119,CA-2014-101434,Consumer,Technology,Accessories,Razer Kraken PRO Over Ear PC and Music Headset,TEC-AC-10002402,United States,US,East,...,71.9910,TR-21325,Toby Ritter,Medium,7109.0,Standard Class,16.95,6/1/2014,239.970,167.9790
39812,CA-2014-39996,CA-2014-119284,Corporate,Technology,Phones,Samsung Galaxy S III - 16GB - pebble blue (T-M...,TEC-PH-10000560,United States,US,South,...,223.9936,TS-21205,Thomas Seio,Medium,37211.0,Standard Class,114.65,6/1/2014,2239.936,2015.9424
39813,CA-2014-39993,CA-2014-119284,Corporate,Technology,Phones,HTC One,TEC-PH-10001051,United States,US,South,...,26.9973,TS-21205,Thomas Seio,Medium,37211.0,Standard Class,11.73,6/1/2014,239.976,212.9787


**Notice that we go from 39,828 rows to 3319 rows**

<a id='pivot'></a>

#### Create a wide dataframe with .pivot()
   
**<code>[.pivot()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html)</code>** allows us to transform rows into columns
    
<img src="../support_files/images/pandas/pandas_pivot.png">  

Example: 
<code>df.pivot(columns = [ column(s) ])</code>

Parameters:    

* <code>columns</code>: Column to use to make new frame’s columns.
* <code>index</code> (optional): Column to use to make new frame’s index. If None, uses existing index.
* <code>values</code> (optional): Column(s) to use for populating new frame’s values. If not specified, all remaining columns will be used and the result will have hierarchically indexed columns.   

<img src="../support_files/images/pandas/pandas_pivot_b.png" width ='550'>


In [42]:
# get just the 'state' and 'Category' columns and turn the 'Category' column into rows 
USA_COGS_wide = usa_sales_long.pivot(index = ['Row ID'],
                                     columns='Order Date',
                                     values = 'Sales')

# view the resulting dataframe.
USA_COGS_wide

Order Date,1/1/2014,10/1/2014,11/1/2014,12/1/2014,2/1/2014,3/1/2014,4/1/2014,5/1/2014,6/1/2014,7/1/2014,8/1/2014,9/1/2014
Row ID,,,,,,,,,,,,
CA-2014-31309,NaN,NaN,NaN,NaN,NaN,NaN,15.552,NaN,NaN,NaN,NaN,NaN
CA-2014-31331,NaN,29.472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA-2014-31338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.168
CA-2014-31340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.616
CA-2014-31368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.050
...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-41126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.352,NaN,NaN
US-2014-41173,NaN,NaN,NaN,NaN,NaN,NaN,848.544,NaN,NaN,NaN,NaN,NaN
US-2014-41174,NaN,NaN,NaN,NaN,NaN,NaN,8.700,NaN,NaN,NaN,NaN,NaN


<a id='crosstab'></a>

#### Aggregate and Get Frequency Counts

 **<code>[.crosstab()](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html)</code>** allows you to aggregate by category and get frequency counts. 
    
Cross-tabulations or contingency tables are tables used to describe relationships between two categorical variables. The table displays the frequency distribution of the variables. This is a very easy built in function to use, but the downside is that it is not flexible. It only offers frequency counts. 

<code>x_tab = pd.crosstab(index = df[categorical column], columns=df[categorical column])</code>
* <code>index</code> Values to group by in rows
* <code>columns</code> Values to group by in columns


In [43]:
# In this example we want to be able to count number of orders
# by order priority and shipping mode

orders_xtab = pd.crosstab(USA_long['Order Priority'], 
                   columns = USA_long['Ship Mode'])

# view the results
orders_xtab

Ship Mode,First Class,Same Day,Second Class,Standard Class
Order Priority,,,,
Critical,1368,516,732,0
High,3336,1164,3288,5244
Low,0,0,0,1956
Medium,2172,552,3864,15636


<a id='pivot_table'></a>

#### Aggregate data using .pivot_table()

**<code>[.pivot_table()](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)</code>** allows us to construct quick aggregate tables using categorical variables. This is slightly more complicated than crosstab but it offers more flexibility in aggregator functions.
    
Example: 
<code>pd.pivot_table(df, values = [ ], index = [ ], columns = [ ], aggfunct = )</code>
    
* <code>values</code>: The column to aggregate (if blank, will aggregate all numerical values)
* <code>index</code>: The column or columns to group data by. A single column can be a string, while multiple columns should be a list of strings
* <code>columns</code>: The column or columns to group data by. A single column can be a string, while multiple columns should be a list of strings
* <code>aggfunc</code>: the type of aggregator function to use, you can use one or many
    * Examples: 'count', 'Mean': np.mean, 'Sigma': np.std


In [44]:
# To see how this works lets make something similar to our crosstab with counts! 

# Lets use Order ID as our values because it's unique
# set our index to "Order Priority"
# use "Ship Mode" as our columns
# and lets count as our aggregate function to count how many 
# Order IDs there are for each order priorty and shipping mode

pd.pivot_table(USA_long,
               values  = ['Order ID'],
               index   = ['Order Priority'],
               columns = ['Ship Mode'],
               aggfunc = ['count'])

count                                     
                  Order ID                                     
Ship Mode      First Class Same Day Second Class Standard Class
Order Priority                                                 
Critical            1368.0    516.0        732.0            NaN
High                3336.0   1164.0       3288.0         5244.0
Low                    NaN      NaN          NaN         1956.0
Medium              2172.0    552.0       3864.0        15636.0

In [45]:
# Lets do the same thing but with columns that contain values rather than categories. 
# Lets use Sales as our values, and Order Priority and Segment as our index and columns
# we can also get sum and mean at the same time!

pd.pivot_table(USA_long, 
               values  = ['Sales'],
               index   = ['Order Priority'],
               columns = ['Segment'],
               aggfunc = [np.sum, np.mean])

sum                                 mean              \
                      Sales                                Sales               
Segment            Consumer    Corporate Home Office    Consumer   Corporate   
Order Priority                                                                 
Critical         15523.3562   13242.7100  12487.2920  147.841488  217.093607   
High            103756.7947   81056.0856  68417.2380  185.944077  263.169109   
Low              12470.4220   12673.7510   7726.4540  148.457405  264.036479   
Medium          200722.4870  135038.6858  70831.7469  216.762945  239.006524   

                            
                            
Segment        Home Office  
Order Priority              
Critical        240.140231  
High            310.987445  
Low             249.240452  
Medium          196.209825

<a id='groupby'></a>

#### Grouping data by category with .groupby()

**<code>[.groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html</code>)** allows you to group a dataframe using a mapper or by a Series of columns.This can be used to group large amounts of data and compute operations on these groups.


<code> grouped_object = df.groupby(by =[columns])</code>

<img src="../support_files/images/pandas/pandas_groupby_object.png" width ='70%'>  
    
Groupby objects are not viewable the same way regular dataframes are. When you create a groupby object in a jupyter notebook the output will look something like so:
<img src="../support_files/images/pandas/pandas_groupby_object_output.png" width ='70%'>  
    
To access and view individual dataframes/groups from the groupby object you can use 
    **<code>[.get_group()](https://pandas.pydata.org/pandas-docs/version/1.3.2/reference/api/pandas.core.groupby.GroupBy.get_group.html</code>)**
* <code>grouped_object.get_group('group')</code>

    
<img src="../support_files/images/pandas/pandas_groupby_data_access.png" width ='60%'> 



In [46]:
# Lets practice creating a groupby object.

# first lets subset usa sales long and group it by region
region_agg = usa_sales_long[['Region', 'Category','COGS']].groupby('Region')

# then we can view the dataframe of one of our region groups
region_agg.get_group('Central')

,Region,Category,COGS
55,Central,Furniture,194.2044
98,Central,Furniture,420.1578
122,Central,Furniture,109.2978
123,Central,Furniture,1083.0670
425,Central,Furniture,286.7226
...,...,...,...
39473,Central,Technology,68.0157
39728,Central,Technology,78.1868
39737,Central,Technology,41.9860
39800,Central,Technology,352.9578


#### Aggregating and Statistics with groupby
    
You can run analytics or further aggregations on each of the groups in your groupby object.

A groupby aggregation operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups. You can use many of the descriptive statistics functions we talked about early in the notebook or **<code>[.agg()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)</code>** will also compute aggregation on the grouped object.  Example aggregators: min, max, sum, mean, median etc. 

* <code>grouped_object.max()</code>
* <code>grouped_object.agg([aggregator])</code>
    

**Groupby "pipeline":**<br>
<img src="../support_files/images/pandas/pandas_groupby_pipeline.png">      


In [47]:
# This last is gonna be slightly more complicated! 

# Now lets subset the dataframe to just "Region" and "COGS"
COGS_df = usa_sales_long[['Region', 'COGS']]

# Groupby Region and get the sum for each region
region_cogs = COGS_df.groupby(by = 'Region').agg(['sum'])

# Lets view the output dataframe
region_cogs

,COGS
,sum
Region,
Central,139547.2840
East,180044.0698
South,114116.2564
West,206731.8999


#### Adding aggregated data back to original dataframe
    
There may be cases where you don't want a whole new dataframe but would just like to add a column of aggregated data back to your original dataframe. In that case the  **<code>[.transform()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html)</code>** function may be useful to you. 
    
<code>df('new_column') = df.groupby(by='category_column')[['data_column']].transform(statistical_function)</code>

**Groupby "pipeline":**<br>
<img src="../support_files/images/pandas/pandas_groupby_transform.png">      


In [48]:
# Final Example! 

# Lets do a similar thing as above (getting cost of goods by reion) but put it back in the original dataframe
usa_sales_long['regional_COGS'] = usa_sales_long.groupby(by = 'Region')[['COGS']].transform(np.sum)

# Lets view the output dataframe (subset it so we can see the relevant columns)
usa_sales_long[["Region", "COGS", "regional_COGS"]]

,Region,COGS,regional_COGS
18,East,160.1168,180044.0698
19,East,20.3000,180044.0698
21,East,254.5008,180044.0698
22,East,765.4752,180044.0698
38,West,177.7113,206731.8999
...,...,...,...
39801,Central,407.9400,139547.2840
39811,East,167.9790,180044.0698
39812,South,2015.9424,114116.2564
39813,South,212.9787,114116.2564


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise 8.7**: What State has the highest Profit in the US?

**Exercise 8.8**: Which Customer had the 3rd highest total Sales?

HINT: use the index to easily access sorted data by location</div>


In [49]:
# Answer 7.7
# aggregate the correct data & sort it
profit_by_state = USA_long[['State', 'Profit']].groupby('State').agg(['sum'])
profit_by_state.sort_values(by=[('Profit', 'sum')], ascending = False, inplace=True)

# print state with highest profit
print("Most profitable state: " + profit_by_state.index[0])

Most profitable state: California


In [50]:
# Answer 7.8

# same process as above
sales_by_cust = USA_long[['Customer Name', 'Sales']].groupby('Customer Name').agg(['sum'])
sales_by_cust.sort_values(by=[('Sales', 'sum')], ascending = False, inplace=True)

# get the 3rd row
print("Customer with 3rd highest total sales: " + sales_by_cust.index[2])

Customer with 3rd highest total sales: Hunter Lopez


<a id='table'></a>

## Creating, Joining and Saving dataframes


**<a href='#create_table'> Creating DataFrames from Scratch:</a>** 
* <a href='#table_from_array'> From an array</a>
* <a href='#table_from_lists'> From a lists</a>
* <a href='#table_from_dict'> From a dictionary</a>
 

**Combining DataFrames**
* <a href='#concat'>Append tables with the same structure using .concat</a>   
* <a href='#reset_index'>Resetting an index</a> 
* <a href='#merge_df'>Merging dataFrames</a>
* <a href='#merge_left_right'>Left and right merges</a>
* <a href='#merge_inner_outer'>Inner and outer merges</a>

**Saving DataFrames**
* <a href='#save_df'>Save to excel</a>
* <a href='#save_df'>Save to .csv</a>


<a id='create_table'></a>

#### Creating a DataFrame

You can create dataframes from scratch by using **<code>[pd.DataFrame()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)</code>**

    
Because dataFrames are tabular, your data will need column names. This usually means that creating a dataframe from scratch will require some combination of using arrays, lists or dictionaries. 


<a id='table_from_array'></a>

#### Create a dataframe from a numpy array

When creating a dataframe from an array it's important that you know what each dimension of your array represents. Lists and dictionaries are a little more self-documenting in that way, but using arrays can be very efficient. 

You must always provide the correct number of column labels to match your array dimensions 
* <code>pd.DataFrame(data_array, columns=['column_1','column_2'...])</code>
    
     
Note: for this example we will have to create a numpy array (using <code>numpy.random.rand</code>), however in the real world you would already have your array that you'd like to turn into a dataframe.


In [51]:
# first create the array using numpy random.rand(rows,columns)
data_array = np.random.rand(25,3) 
data_array

array([[0.9234395 , 0.03300932, 0.90926161],
       [0.93688426, 0.98780622, 0.05223736],
       [0.00189885, 0.79378243, 0.01103786],
       [0.76688821, 0.20287324, 0.1982282 ],
       [0.60916514, 0.24471379, 0.1300328 ],
       [0.1243073 , 0.12690162, 0.6210577 ],
       [0.11479377, 0.93064953, 0.25473702],
       [0.45278532, 0.61660623, 0.88313392],
       [0.93317576, 0.77945232, 0.19370775],
       [0.43079175, 0.75747082, 0.27507852],
       [0.40937412, 0.98995794, 0.39306526],
       [0.84248669, 0.52538166, 0.74230099],
       [0.94037133, 0.11424214, 0.45516923],
       [0.63333198, 0.95851856, 0.45040759],
       [0.39483367, 0.40191674, 0.66515693],
       [0.18825632, 0.40485814, 0.91168511],
       [0.10838877, 0.64143315, 0.30658834],
       [0.03567433, 0.02123722, 0.20576998],
       [0.99857754, 0.16302413, 0.10658032],
       [0.95976907, 0.32901725, 0.84641052],
       [0.17301841, 0.49606215, 0.18647609],
       [0.54477554, 0.60562417, 0.37041347],
       [0.

In [52]:
# create the dataframe, assigning data first and then the columns
# our array is 25 rows x 3 columns so we must provide 3 column labels
array_df = pd.DataFrame(data_array, columns=['column_1','column_2','column_3'])

#view the dataframe
array_df

,column_1,column_2,column_3
0,0.923439,0.033009,0.909262
1,0.936884,0.987806,0.052237
2,0.001899,0.793782,0.011038
3,0.766888,0.202873,0.198228
4,0.609165,0.244714,0.130033
5,0.124307,0.126902,0.621058
6,0.114794,0.930650,0.254737
7,0.452785,0.616606,0.883134
8,0.933176,0.779452,0.193708
9,0.430792,0.757471,0.275079


<a id='table_from_lists'></a>

#### Dataframe from lists

You can easily make a dataframe from multiple lists and a dictionary. 
* Each list represents a column of data.
* Dictionary keys represent column names
* Dictionary values are the lists that contain all the data/rows  
    
Note: all lists must be the same length!



For the lists and dictionaries examples we will make a dataframe of adoptable pets!


In [53]:
# first make the lists, we will have 4 entries for each list
name = ['Oreo','Squid','Mrs Noris', 'Dazzler']
age_months = [5, 18, 12, 9]


# then assign the lists to appropriate column names using dictionary keys
dict_of_lists = {'name': name,
                 'age_months': age_months}

# finally create a dataframe using the dictionary
dict_df1 = pd.DataFrame(dict_of_lists)
dict_df1

,name,age_months
0,Oreo,5
1,Squid,18
2,Mrs Noris,12
3,Dazzler,9


In [54]:
# Another way to make a dataframe using lists and dictionaries is to do the dictionary
# assignment within the dataframe creation call. This just cuts out a middle step

names = ['Cheddar', 'Zuko']
age_months = [7, 21]

dict_df2 = pd.DataFrame({'name':names,
                         'age_months': age_months})
dict_df2

,name,age_months
0,Cheddar,7
1,Zuko,21


<a id='table_from_dict'></a>

#### Dataframe from list of dictionaries

You can also create a single list that contains multiple dictionaries. 
* Each dictionary represents a row
* Each key represents a column. 
 
Note: you must use the exact same keys in all dictionaries using this method.


In [55]:
list_of_dicts = [
    {'name': 'Oreo',      'species': 'dog'},
    {'name': 'Dazzler',   'species': 'cat'},
    {'name': 'Templeton', 'species': 'rat'},
]

list_df = pd.DataFrame(list_of_dicts)
list_df

,name,species
0,Oreo,dog
1,Dazzler,cat
2,Templeton,rat


<a id='concat'></a>

#### Append dataframes using concat

You can combine two dataframes by appending one dataframe to another if they have the same structure(i.e. same columns OR same indexes) using **<code>[.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)</code>**

concatenate rows:
    <code>pd.concat([df1,df2])</code>
  
<img src="../support_files/images/pandas/pandas_concat_rows.png">    

concatenate columns:
    <code>pd.concat([df1,df2], axis=1)</code>

<img src="../support_files/images/pandas/pandas_concat_columns.png"> 



In [56]:
# dict_df1 and dict_df2 have the same structure without redundancy
# so they're good candidates for concatenation. 

pd.concat([dict_df1, dict_df2])


,name,age_months
0,Oreo,5
1,Squid,18
2,Mrs Noris,12
3,Dazzler,9
0,Cheddar,7
1,Zuko,21


In [57]:
# Lets try concatinating again but resetting the index while we do so

pd.concat([dict_df1, dict_df2]).reset_index(drop=True)

,name,age_months
0,Oreo,5
1,Squid,18
2,Mrs Noris,12
3,Dazzler,9
4,Cheddar,7
5,Zuko,21


<a id='merge_df'></a>

## Merging DataFrames

Needing to merge or join dataframes is a very common occurance in data analysis. Frequently one table contains a specific type of data while another table contains some other data, and you wish to combine that information. 

There are several different ways to merge DataFrames. The most common way is using **<code>[.merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)</code>**

Important function arguments:
* <code>how</code> determines the way the dataframes are merged. There are 4 basic options: 
    * <code>'left'</code> & <code>'right'</code>
    * <code>'inner'</code>(default) & <code>'outer'</code>
    
* <code>on</code> argument identifies which column(s) are common across both dataframes and should be used as the index to merge the dataframes on
    * in the example images below 'x1' is the common column that would be used
        * <code>on ='x1'</code>


<a id='merge_left_right'></a>

#### Left and Right merges

For <code>'left'</code> and <code>'right'</code> merges, the order that you list the dataframes in the <code>.merge()</code> function matters. The first dataframe listed is the 'left' dataframe and the second is the 'right' dataframe.

* left merge:
    * Joins matching rows from df_right to df_left, retaining all the information from the df_left.
    * <code>left_merge_df = pd.merge(df_left, df_right, how='left', on='x1')</code>
  
<img src="../support_files/images/pandas/pandas_join_left.png">    

* right merge example: 
    * Joins matching rows from df_left to df_right, retaining all the information from the df_right
    * <code>right_merge_df = pd.merge(left_df, right_df, how='right', on='x1')</code>

<img src="../support_files/images/pandas/pandas_join_right.png"> 


In [58]:
# Lets do a LEFT join 
# dict_df1 which contains information on animal 'name' and 'age_months'
# list_df contains 'name' and 'species'

pd.merge(dict_df1, list_df, how = 'left', on ='name')

,name,age_months,species
0,Oreo,5,dog
1,Squid,18,NaN
2,Mrs Noris,12,NaN
3,Dazzler,9,cat


In [59]:
# Try a right join with the same dataframes! 

pd.merge(dict_df1, list_df, how = 'right', on ='name')

,name,age_months,species
0,Oreo,5.0,dog
1,Dazzler,9.0,cat
2,Templeton,NaN,rat


<a id='merge_inner_outer'></a>

#### Inner and Outer Merges

For <code>'inner'</code> and <code>'outer'</code> merges, the order of the dataframes does not matter. Instead the inner and outer merges are based on what is common between both dataframes.

* inner merge:
    * Join data and retains only rows found in both dataframes.
    * <code>inner_merge_df = pd.merge(df1, df2, how='inner', on='x1')</code>
  
<img src="../support_files/images/pandas/pandas_join_inner.png">    

* outer merge: 
    * Joins dataframes and retains all values, all rows.
    * <code>outer_merge_df = pd.merge(df1, df2, how='outer', on='x1')</code>

<img src="../support_files/images/pandas/pandas_join_outer.png">


In [60]:
# Lets try an outer merge with those same dataframes

pd.merge(dict_df1, list_df, how = 'outer', on ='name')

,name,age_months,species
0,Oreo,5.0,dog
1,Squid,18.0,NaN
2,Mrs Noris,12.0,NaN
3,Dazzler,9.0,cat
4,Templeton,NaN,rat


#### Merging with Indexes
    
You may have two tables that you'd like to merge where the index of one dataframe is a column in another dataframe. 
    
<img src="../support_files/images/pandas/pandas_index_merge.png">  

<code>df1.merge(df2, left_on = "x3", right_index = True)</code>    

**Parameters:**

<code>left_on</code> label or list, or array-like
Column or index level names to join on in the left DataFrame. Can also be an array or list of arrays of the length of the left DataFrame. These arrays are treated as if they are columns.

<code>right_on</code> label or list, or array-like
Column or index level names to join on in the right DataFrame. Can also be an array or list of arrays of the length of the right DataFrame. These arrays are treated as if they are columns.

<code>left_index</code> bool, default False
Use the index from the left DataFrame as the join key(s). If it is a MultiIndex, the number of keys in the other DataFrame (either the index or a number of columns) must match the number of levels.

<code>right_index</code> bool, default False
Use the index from the right DataFrame as the join key. Same caveats as left_index.

NOTE: If you have other columns in common besides the merging column/index these will appear twice in your new dataframe!


<a id='save_df'></a>

#### Saving dataframes

Just like loading dataframes, there are many different formats that you can save a dataframe to. We will just show you .csv and .xls but to see a complete list please check out the [input/output documentation here](https://pandas.pydata.org/docs/reference/io.html):

* [.to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)
    * <code>df.to_csv('df_save_name.csv')</code>

* [.to_excel()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html#pandas.DataFrame.to_excel)
    * <code>df.to_excel('df_save_name.xlsx')</code>

    
by default the save functions will save to your working directory. You must specify a path if you wish to save your dataframe elsewhere.


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise 8.9**: Read in population data from the csv file and show the columns (find file in: ../support_files/datasets/population_by_country_2020.csv).

**Exercise 8.10**: Perform a left merge on the original dataframe (df) and new population population dataframe, using "Country" column, print columns to confirm merge

**Exercise 8.11**: Return the merged dataframe from "Country" with the highest population, first 10 rows

This country population dataset was provided mostly as an example for you to trying merging two diffeferent datasets. But feel free to consider interesting questions and corresponding plots to investigate this merged data. For example, is there a difference in categories of items sold in high vs low population countries?</div>


In [61]:
# Answer 7.9:
filepath = os.path.join('support_files', 'datasets', 'population_by_country_2020.csv')

pop_df = pd.read_csv(filepath)

print(pop_df.columns)

Index(['Country', 'Population', 'Yearly Change', 'Net Change',
       'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)', 'Fert. Rate',
       'Med. Age', 'Urban Pop %', 'World Share'],
      dtype='object')


In [62]:
# Answer 7.10:
merged_df  = pd.merge(df, pop_df, how = 'left', on ='Country')
print(merged_df.columns)

Index(['Order ID', 'Segment', 'Category', 'Sub-Category', 'Product Name',
       'Product ID', 'Country', 'Market', 'Region', 'Quantity', 'Discount',
       'Profit', 'Customer ID', 'Customer Name', 'Order Priority',
       'Postal Code', 'Ship Mode', 'Shipping Cost', '10/1/2014', '7/1/2014',
       '11/1/2014', '9/1/2014', '1/1/2014', '12/1/2014', '8/1/2014',
       '5/1/2014', '3/1/2014', '4/1/2014', '2/1/2014', '6/1/2014', 'City',
       'State', 'manufacturers', 'Profit Per Unit', 'manufacturers_count',
       'Population', 'Yearly Change', 'Net Change', 'Density (P/Km²)',
       'Land Area (Km²)', 'Migrants (net)', 'Fert. Rate', 'Med. Age',
       'Urban Pop %', 'World Share'],
      dtype='object')


In [63]:
# Answer 7.11:
merged_df.sort_values(by=['Population'], ascending = False).head(10)
merged_df

,Order ID,Segment,Category,Sub-Category,Product Name,Product ID,Country,Market,Region,Quantity,...,Population,Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,IN-2014-75456,Consumer,Furniture,Furnishings,"Rubbermaid Door Stop, Erganomic",FUR-FU-10004064,Afghanistan,APAC,Central Asia,2,...,39074280.0,2.33 %,886592.0,60.0,652860.0,-62920.0,4.6,18,25 %,0.50 %
1,IN-2014-29767,Home Office,Furniture,Bookcases,"Ikea Library with Doors, Mobile",FUR-BO-10001255,Afghanistan,APAC,Central Asia,2,...,39074280.0,2.33 %,886592.0,60.0,652860.0,-62920.0,4.6,18,25 %,0.50 %
2,IN-2014-29767,Home Office,Furniture,Furnishings,"Rubbermaid Door Stop, Erganomic",FUR-FU-10004064,Afghanistan,APAC,Central Asia,4,...,39074280.0,2.33 %,886592.0,60.0,652860.0,-62920.0,4.6,18,25 %,0.50 %
3,IN-2014-20415,Home Office,Furniture,Bookcases,"Bush Classic Bookcase, Pine",FUR-BO-10002204,Afghanistan,APAC,Central Asia,5,...,39074280.0,2.33 %,886592.0,60.0,652860.0,-62920.0,4.6,18,25 %,0.50 %
4,IN-2014-47337,Corporate,Furniture,Chairs,"Hon Rocking Chair, Red",FUR-CH-10003965,Afghanistan,APAC,Central Asia,7,...,39074280.0,2.33 %,886592.0,60.0,652860.0,-62920.0,4.6,18,25 %,0.50 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17526,ZA-2014-9750,Corporate,Technology,Accessories,"Memorex Router, USB",TEC-MEM-10002202,Zambia,Africa,Africa,1,...,18468257.0,2.93 %,522925.0,25.0,743390.0,-8000.0,4.7,18,45 %,0.24 %
17527,ZI-2014-7610,Corporate,Technology,Machines,"StarTech Phone, Red",TEC-STA-10000699,Zimbabwe,Africa,Africa,1,...,14899771.0,1.48 %,217456.0,38.0,386850.0,-116858.0,3.6,19,38 %,0.19 %
17528,ZI-2014-5970,Home Office,Technology,Accessories,"Belkin Router, USB",TEC-BEL-10003985,Zimbabwe,Africa,Africa,1,...,14899771.0,1.48 %,217456.0,38.0,386850.0,-116858.0,3.6,19,38 %,0.19 %
17529,ZI-2014-9550,Consumer,Technology,Machines,"Konica Receipt Printer, Red",TEC-KON-10003116,Zimbabwe,Africa,Africa,2,...,14899771.0,1.48 %,217456.0,38.0,386850.0,-116858.0,3.6,19,38 %,0.19 %
